### Loading the dataset

In [1]:
import pandas as pd

df = pd.read_csv("data/movie_data.csv")
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [2]:
df["review"][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [3]:
df.shape

(50000, 2)

## <h2 align="center">Bag of words / Bag of N-grams model</h2>

### Transforming documents into feature vectors

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()



docs = np.array(['The sun is shining',
'The weather is sweet',
'The sun is shining, the weather is sweet, and one and one is two'])
#the above statement is converted to array and then to a list of words.

bag = count.fit_transform(docs)

In [5]:
print(count.vocabulary_)
#it returns the unique words in the sentence and their index in the dictionary

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
print(bag.toarray()) 
#it represents the occurence of words in the three sentences.
#columns represent the unique words occurence in the sentences
#first column is of and second is of is and so on as per the index in the dictionary above

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*

### Word relevancy using term frequency-inverse document frequency

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf = True,norm = "l2",smooth_idf = True) #smooth_idf prevents devision by zero
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision = 2) #prints to decimal places
tfidf = TfidfTransformer(use_idf = True,norm = "l2",smooth_idf = True) #smooth_idf prevents devision by zero
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


### Data Preparation

In [9]:
df["review"][0][-50:] #to print last 50 characters

'is seven.<br /><br />Title (Brazil): Not Available'

In [10]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #to replace html tags with space
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) #finding emoticons
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '') #to move founded emoticons to end of review
    return text

In [11]:
preprocessor(df["review"][0][-50:]) #html tags removed

'is seven title brazil not available'

In [12]:
preprocessor("</a>this:) is a :( test :-)!")  #emojis moved to the end 
#emojis are moved to end and

'this is a test :) :( :)'

In [13]:
df["review"] = df["review"].apply(preprocessor)

### Tokenization of documents

In [24]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

In [17]:
def tokenizer(text):
    return text.split()

In [22]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [18]:
tokenizer("runners like running thus they run")

['runners', 'like', 'running', 'thus', 'they', 'run']

In [25]:
tokenizer_porter("runners like running thus they run")

['runner', 'like', 'run', 'thu', 'they', 'run']

In [26]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/rhyme/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
from nltk.corpus import stopwords

stop= stopwords.words("english")
[w for w in tokenizer_porter("a running like running and runs a lot")[-10:] if w not
in stop]

['run', 'like', 'run', 'run', 'lot']

### Transform Text Data into TF-IDF Vectors

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False,
                       preprocessor = None,
                       tokenizer = tokenizer_porter,
                       use_idf = True,
                       norm = "l2",
                       smooth_idf = True)

y = df.sentiment.values
X = tfidf.fit_transform(df.review)

### Document Classification using Logistic Regression

In [32]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 1,test_size = 0.5,
                                                shuffle = False)

In [33]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv = 5,
                          scoring = "accuracy",
                          random_state = 0,
                          n_jobs = -1,
                          verbose = 3,
                          max_iter = 300).fit(X_train,y_train)

saved_model = open("saved_model.sav","wb")
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.5min finished


###  Model Evaluation

In [34]:
filename = "saved_model.sav"
saved_clf = pickle.load(open(filename,"rb"))

In [35]:
saved_clf.score(X_test,y_test)

/home/rhyme/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1926: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.89604